In [1]:
import sys
sys.path.append('/Users/erikweilandt/Library/Caches/pypoetry/virtualenvs/tepem-FeojsIC2-py3.11/lib/python3.11/site-packages')

In [2]:
from sympy import symbols, simplify, prod, plot, diff
import numpy as np

In [3]:
def LagrangPoly(y,order,i,yi=None):
    if yi==None:
        yi=symbols('y:%d'%(order+1))
    index = list(range(order+1))
    index.pop(i)
    return prod([(y-yi[j])/(yi[i]-yi[j]) for j in index])

reference: https://www.longqi.cf/python/2014/03/24/implement-of-lagrange-polynomial-in-sympy/

In [4]:
test_order = 2
test_nodes = np.linspace(0,1, test_order+1)

In [5]:
x, y=symbols('x y')
LagrangPoly(y,test_order,0)

(y - y1)*(y - y2)/((y0 - y1)*(y0 - y2))

## Define 1-D $\mathbb{P}_d$ shape functions in both directions

In [6]:
order_x = 1
order_y = 4

In [7]:
nodes_x = np.linspace(0,1, order_x+1)
nodes_y = np.linspace(0,1, order_y+1)

In [8]:
pols_x = {}
for i in range(order_x+1):
    pols_x[f"psi_{i}"] = simplify(LagrangPoly(x,order_x,i,list(nodes_x)))
pols_y = {}
for i in range(order_y+1):
    pols_y[f"xi_{i}"] = simplify(LagrangPoly(y,order_y,i,list(nodes_y)))
pols_x, pols_y

({'psi_0': 1.0 - 1.0*x, 'psi_1': 1.0*x},
 {'xi_0': 1.0*(y - 1)*(1.33333333333333*y - 1.0)*(2.0*y - 1.0)*(4.0*y - 1.0),
  'xi_1': y*(-42.6666666666667*y**3 + 96.0*y**2 - 69.3333333333333*y + 16.0),
  'xi_2': 4.0*y*(y - 1)*(4.0*y - 3.0)*(4.0*y - 1.0),
  'xi_3': y*(-42.6666666666667*y**3 + 74.6666666666667*y**2 - 37.3333333333333*y + 5.33333333333333),
  'xi_4': y*(10.6666666666667*y**3 - 16.0*y**2 + 7.33333333333333*y - 1.0)})

In [9]:
#for pol in pols_x.values():
#    plot(pol, (x,0,1))

# $\mathbb{Q}_{x,y}$ shape functions

Below we are numerating such that we follow the numbering as indicated in the notes. This means we first number all shape functions for x=0 and at last all shape functions with x=1.

In [10]:
q_sf = {}
name_counter = 0
for x_sf in pols_x.values():
    for y_sf in pols_y.values():
        q_sf[f"phi_{name_counter}"] = x_sf * y_sf
        name_counter += 1
q_sf

{'phi_0': 1.0*(1.0 - 1.0*x)*(y - 1)*(1.33333333333333*y - 1.0)*(2.0*y - 1.0)*(4.0*y - 1.0),
 'phi_1': y*(1.0 - 1.0*x)*(-42.6666666666667*y**3 + 96.0*y**2 - 69.3333333333333*y + 16.0),
 'phi_2': 4.0*y*(1.0 - 1.0*x)*(y - 1)*(4.0*y - 3.0)*(4.0*y - 1.0),
 'phi_3': y*(1.0 - 1.0*x)*(-42.6666666666667*y**3 + 74.6666666666667*y**2 - 37.3333333333333*y + 5.33333333333333),
 'phi_4': y*(1.0 - 1.0*x)*(10.6666666666667*y**3 - 16.0*y**2 + 7.33333333333333*y - 1.0),
 'phi_5': 1.0*x*(y - 1)*(1.33333333333333*y - 1.0)*(2.0*y - 1.0)*(4.0*y - 1.0),
 'phi_6': 1.0*x*y*(-42.6666666666667*y**3 + 96.0*y**2 - 69.3333333333333*y + 16.0),
 'phi_7': 4.0*x*y*(y - 1)*(4.0*y - 3.0)*(4.0*y - 1.0),
 'phi_8': 1.0*x*y*(-42.6666666666667*y**3 + 74.6666666666667*y**2 - 37.3333333333333*y + 5.33333333333333),
 'phi_9': 1.0*x*y*(10.6666666666667*y**3 - 16.0*y**2 + 7.33333333333333*y - 1.0)}

In [11]:
q_sf_velo = {}
name_counter = 0
for i, x_sf in enumerate(pols_x.values()):
    for j, y_sf in enumerate(pols_y.values()):
        if j%order_y != 0:
            q_sf_velo[f"phi_{name_counter}"] = x_sf * y_sf
            name_counter += 1
q_sf_velo

{'phi_0': y*(1.0 - 1.0*x)*(-42.6666666666667*y**3 + 96.0*y**2 - 69.3333333333333*y + 16.0),
 'phi_1': 4.0*y*(1.0 - 1.0*x)*(y - 1)*(4.0*y - 3.0)*(4.0*y - 1.0),
 'phi_2': y*(1.0 - 1.0*x)*(-42.6666666666667*y**3 + 74.6666666666667*y**2 - 37.3333333333333*y + 5.33333333333333),
 'phi_3': 1.0*x*y*(-42.6666666666667*y**3 + 96.0*y**2 - 69.3333333333333*y + 16.0),
 'phi_4': 4.0*x*y*(y - 1)*(4.0*y - 3.0)*(4.0*y - 1.0),
 'phi_5': 1.0*x*y*(-42.6666666666667*y**3 + 74.6666666666667*y**2 - 37.3333333333333*y + 5.33333333333333)}

### Testing numeration

In [12]:
# check if the correct nodes are == 1
for i in range(order_y-1):
    assert q_sf_velo[f"phi_{i}"].subs([(x,0), (y,nodes_y[i+1])]) < 1+10e-10 and q_sf_velo[f"phi_{i}"].subs([(x,0), (y,nodes_y[i+1])]) > 1-10e-10, f"Node {i} is wrong"

In [13]:
grad_q = {}
for i, sf in enumerate(q_sf.values()):
    grad_x = diff(sf, x)
    grad_y = diff(sf, y)
    grad_q[f"grad_phi_{i}"] = (grad_x, grad_y)
#grad_q

In [14]:
grad_q_velo = {}
for i, sf in enumerate(q_sf_velo.values()):
    grad_x = diff(sf, x)
    grad_y = diff(sf, y)
    grad_q_velo[f"grad_phi_{i}"] = (grad_x, grad_y)
#grad_q_velo

In [15]:
# Printing all shape functions
for sf_name, sf in q_sf.items():
    def_header = f"def {sf_name}(x: float, y: float) -> float:\n"
    body = f"\treturn {sf}"
    #print(def_header + body)

In [16]:
# Printing only velocity shape functions: leaving out the boundary shape functions
for sf_name, sf in q_sf_velo.items():
    def_header = f"def {sf_name}(x: float, y: float) -> float:\n"
    body = f"\treturn {sf}"
    print(def_header + body)

def phi_0(x: float, y: float) -> float:
	return y*(1.0 - 1.0*x)*(-42.6666666666667*y**3 + 96.0*y**2 - 69.3333333333333*y + 16.0)
def phi_1(x: float, y: float) -> float:
	return 4.0*y*(1.0 - 1.0*x)*(y - 1)*(4.0*y - 3.0)*(4.0*y - 1.0)
def phi_2(x: float, y: float) -> float:
	return y*(1.0 - 1.0*x)*(-42.6666666666667*y**3 + 74.6666666666667*y**2 - 37.3333333333333*y + 5.33333333333333)
def phi_3(x: float, y: float) -> float:
	return 1.0*x*y*(-42.6666666666667*y**3 + 96.0*y**2 - 69.3333333333333*y + 16.0)
def phi_4(x: float, y: float) -> float:
	return 4.0*x*y*(y - 1)*(4.0*y - 3.0)*(4.0*y - 1.0)
def phi_5(x: float, y: float) -> float:
	return 1.0*x*y*(-42.6666666666667*y**3 + 74.6666666666667*y**2 - 37.3333333333333*y + 5.33333333333333)


In [17]:
# Printing all gradient shape functions
for sf_name, sf in grad_q.items():
    def_header = f"def {sf_name}(x: float, y: float) -> npt.NDArray[np.float64]:\n"
    body = f"\tphi_dx = {sf[0]}\n\tphi_dy = {sf[1]}\n"
    return_str = f"\treturn np.array([phi_dx, phi_dy])"
    #print(def_header + body + return_str)

In [18]:
# Print velocity dof: leaving out the gradiants of shape function on the boundary
for sf_name, sf in grad_q_velo.items():
    def_header = f"def {sf_name}(x: float, y: float) -> npt.NDArray[np.float64]:\n"
    body = f"\tphi_dx = {sf[0]}\n\tphi_dy = {sf[1]}\n"
    return_str = f"\treturn np.array([phi_dx, phi_dy])"
    print(def_header + body + return_str)

def grad_phi_0(x: float, y: float) -> npt.NDArray[np.float64]:
	phi_dx = -1.0*y*(-42.6666666666667*y**3 + 96.0*y**2 - 69.3333333333333*y + 16.0)
	phi_dy = y*(1.0 - 1.0*x)*(-128.0*y**2 + 192.0*y - 69.3333333333333) + (1.0 - 1.0*x)*(-42.6666666666667*y**3 + 96.0*y**2 - 69.3333333333333*y + 16.0)
	return np.array([phi_dx, phi_dy])
def grad_phi_1(x: float, y: float) -> npt.NDArray[np.float64]:
	phi_dx = -4.0*y*(y - 1)*(4.0*y - 3.0)*(4.0*y - 1.0)
	phi_dy = 16.0*y*(1.0 - 1.0*x)*(y - 1)*(4.0*y - 3.0) + 16.0*y*(1.0 - 1.0*x)*(y - 1)*(4.0*y - 1.0) + 4.0*y*(1.0 - 1.0*x)*(4.0*y - 3.0)*(4.0*y - 1.0) + 4.0*(1.0 - 1.0*x)*(y - 1)*(4.0*y - 3.0)*(4.0*y - 1.0)
	return np.array([phi_dx, phi_dy])
def grad_phi_2(x: float, y: float) -> npt.NDArray[np.float64]:
	phi_dx = -1.0*y*(-42.6666666666667*y**3 + 74.6666666666667*y**2 - 37.3333333333333*y + 5.33333333333333)
	phi_dy = y*(1.0 - 1.0*x)*(-128.0*y**2 + 149.333333333333*y - 37.3333333333333) + (1.0 - 1.0*x)*(-42.6666666666667*y**3 + 74.6666666666667*y**2 - 